# Top N算法
本章节将介绍通过Spark解决Top N问题

我们首先来看一个简单的问题，给定一个不重复的数字列表，求出最大的5个数字。

In [ ]:
rdd = sc.parallelize([3, 8, -1, 19, 14, 27, 1, 5, 11, -2, 88, 34])

In [ ]:
rdd.takeOrdered(5, key=lambda x: -x)

这里我们使用takeOrdered函数筛选出最大的5个元素,其内部以heapq（堆）来实现取最小n个元素，这里我们将key指定为取负数即可变为取最大n个元素

然而实际问题中，我们往往更常见的数据形式是以k-v对出现的，比如URL访问统计、词频统计等
先考虑一个比较简单的情况,假设key是没有重复的：则我们可以对每个分区都求top n，再通过reduce将每个分区的top n规约成最终的top n。
之所以每个分区都要取top n，是因为你不知道最终的top n是如何分布在哪些分区上，甚至可能会有全部属于某个分区的极端情况

那么如果key是有重复的话，情况就不一样了。此时应该需要先做一次wordcount操作保证key不重复，再做top n

在如下的示例中，样例文件*url.txt*每行一个url，共有十多种不同的url，我们将通过Spark计算出现频次最多的前3个

In [ ]:
urls = sc.textFile("url.txt", 4)

In [ ]:
kv = urls.map(lambda x:(x,1)).reduceByKey(lambda x, y: x+y)

In [ ]:
result = kv.takeOrdered(3, key=lambda url_cnt: -url_cnt[1])
result

最终可以看到Spark计算得出了频次最高的3个url：
```
[('facebook.com', 241), ('reddit.com', 166), ('amazon.com', 145)]
```